# Recurrent Neural Networks with Keras

### Introduction
A RNN layer uses a `for` loop to iterate over the timesteps of a sequence, while maintaining an internal state that encodes information about the timestes it has seen so far.

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## Dataset

In [2]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                         as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Shuffling and writing examples to /Users/harshithkumar/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePLB4PD/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/harshithkumar/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePLB4PD/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/harshithkumar/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePLB4PD/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/harshithkumar/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [4]:
encoder = info.features['text'].encoder
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [5]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [7]:
assert original_string == sample_string
for index in encoded_string:
    print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [9]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [10]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

## Model

In [ ]:
model = keras.Sequential([
    layers.Embedding(encoder.vocab_size, 64),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.summary()

## Training

In [17]:
EPOCHS = 10
VALIDAITON_STEPS = 30

In [21]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=EPOCHS,
                   validation_data=test_dataset,
                   validation_steps=VALIDAITON_STEPS)

Epoch 1/10
227/391 [================>.............] - ETA: 2:34 - loss: 0.6924 - accuracy: 0.4983

## Evaluate

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
#Helper function
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_graphs(history, 'accuracy')